# Import Libraries

In [24]:
import sys
import os

notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))

if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [25]:
from text_to_sql import (
    TextToSQL,
    Config,
    LLMConfig,
    SLConfig,
    ContextConfig,
    QueryConfig,
)
from dotenv import load_dotenv
from datetime import datetime

import pandas as pd
import os

# Constants

In [26]:
DATABASE = "academic"
MODEL = "gemini-1.5-pro"
PROVIDER = "gemini"
MAX_RETRIES = 5
RETRY_DELAY = 2

# Load Environment

In [27]:
load_dotenv()

True

# Set Timestamp Experiment

In [28]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
output_dir = f"../files/experiment_result/{timestamp}"
os.makedirs(output_dir, exist_ok=True)

# Config

In [29]:
db_key = DATABASE.upper().replace("-", "_")
provider_key = PROVIDER.upper().replace("-", "_")

config = Config(
    max_retry_attempt=5,
    rewriter_config=LLMConfig(
        type="api",
        model=MODEL,
        provider=PROVIDER,
        api_key=os.getenv(f"API_KEY_{provider_key}"),
    ),
    query_generator_config=LLMConfig(
        type="api",
        model=MODEL,
        provider=PROVIDER,
        api_key=os.getenv(f"API_KEY_{provider_key}"),
    ),
    schema_linker_config=SLConfig(
        type="api",
        model=MODEL,
        provider=PROVIDER,
        api_key=os.getenv(f"API_KEY_{provider_key}"),
        schema_path=f"../files/schema/{DATABASE}.txt",
        metadata_path=f"../files/metadata/{DATABASE}.json",
    ),
    retrieve_context_config=ContextConfig(data_path=f"../files/dataset/dataset_{DATABASE}_example.csv"),
    query_executor_config = QueryConfig(
        host=os.getenv(f"DB_HOST_{db_key}"),
        database=os.getenv(f"DB_DATABASE_{db_key}"),
        user=os.getenv(f"DB_USER_{db_key}"),
        password=os.getenv(f"DB_PASSWORD_{db_key}"),
        port=os.getenv(f"DB_PORT_{db_key}"),
    ),
)

# Model

In [30]:
text_to_sql_model = TextToSQL(config=config)

Initializing API client for gemini using model gemini-1.5-pro.
Initializing API client for gemini using model gemini-1.5-pro.
Initializing API client for gemini using model gemini-1.5-pro.


In [31]:
user_prompt = "Which faculty uses the most facilities?"

# Experiment

In [32]:
# Baseline
query = text_to_sql_model.generate_baseline(user_prompt)
print(query)

Generated SQL Query: SELECT f.nama FROM Fakultas AS f JOIN FasilitasFakultas AS ff ON f.id_fakultas = ff.id_fakultas GROUP BY f.nama ORDER BY COUNT(ff.id_fasilitas) DESC LIMIT 1;

SELECT f.nama FROM Fakultas AS f JOIN FasilitasFakultas AS ff ON f.id_fakultas = ff.id_fakultas GROUP BY f.nama ORDER BY COUNT(ff.id_fasilitas) DESC LIMIT 1;


In [33]:
# Schema Linking
query = text_to_sql_model.predict_sql_schema_only(user_prompt)
print(query)

Entities: ['fakultas', 'fasilitas', 'fasilitasfakultas']
Entity Embeddings: tensor([[-0.0858,  0.0030, -0.1152,  ...,  0.0751,  0.0536,  0.0192],
        [-0.0083,  0.0028, -0.0205,  ...,  0.0032,  0.1153,  0.0406],
        [-0.0313,  0.0165, -0.0624,  ...,  0.0726,  0.1328,  0.0336]],
       device='cuda:0')
Similarity Scores: [('fasilitas', 0.915180429816246), ('fasilitasfakultas', 0.8786247670650482), ('fakultas', 0.7151243090629578), ('keanggotaanklub', 0.574022650718689), ('mahasiswa', 0.552211657166481), ('jurusan', 0.5305024534463882), ('hasilujian', 0.5027420520782471), ('matakuliah', 0.4761573076248169), ('partisipasipenelitian', 0.44340774416923523), ('proyekpenelitian', 0.4387131482362747), ('peminjamanbuku', 0.4334169551730156), ('publikasi', 0.41327446699142456), ('organisasiklub', 0.36894214153289795), ('kelas', 0.32284481450915337), ('catatankehadiran', 0.3000359386205673), ('penulispublikasi', 0.2943468987941742), ('dosen', 0.2878614403307438), ('enrolmenkelas', 0.27840

In [34]:
# Relevant Example
query = text_to_sql_model.predict_sql_with_example_only(user_prompt)
print(query)

Relevant Example: {'relevant_question': 'Which faculty has the most research projects? If there is a tie, show any of the tying faculty. Show the nama and total_proyek.', 'relevant_answer': 'SELECT f.nama, COUNT(p.id_proyek) AS total_proyek\n  FROM Fakultas f\n  JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas\n  GROUP BY f.nama\n  ORDER BY total_proyek DESC\n  LIMIT 1;', 'relevant_summary': "This SQL query identifies the faculty with the highest number of research projects by joining the 'Fakultas' (Faculty) and 'ProyekPenelitian' (Research Projects) tables. It counts projects per faculty and returns the faculty with the most projects, highlighting research activity distribution across faculties. The output helps academic institutions understand research productivity and allocate resources effectively. Key operations include a table JOIN, COUNT aggregation, and ORDER BY with LIMIT to identify the top faculty. The query can be modified to: 1) include time periods to analyze res

In [35]:
# Multistage Generation
query = text_to_sql_model.predict_sql_multistage_only(user_prompt)
print(query)

Generated SQL Query: SELECT f.nama FROM Fakultas f JOIN FasilitasFakultas ff ON f.id_fakultas = ff.id_fakultas GROUP BY f.nama ORDER BY COUNT(ff.id_fasilitas) DESC LIMIT 1;

SELECT f.nama FROM Fakultas f JOIN FasilitasFakultas ff ON f.id_fakultas = ff.id_fakultas GROUP BY f.nama ORDER BY COUNT(ff.id_fasilitas) DESC LIMIT 1;


In [36]:
# Incremental Generation
query = text_to_sql_model.predict_sql_incremental_only(user_prompt)
print(query)

Sub-questions: ['1. List all facilities and the faculty they are associated with.', '2. Count the number of facilities used by each faculty.', '3. Find the maximum number of facilities used by any faculty.', '4.  Identify the faculty (or faculties) that use the maximum number of facilities found in the previous step.']
Generated SQL Query: SELECT f.nama AS nama_fasilitas, fa.nama AS nama_fakultas
FROM Fasilitas AS f
JOIN FasilitasFakultas AS ff ON f.id_fasilitas = ff.id_fasilitas
JOIN Fakultas AS fa ON ff.id_fakultas = fa.id_fakultas;

Generated SQL Query: SELECT f.nama, COUNT(ff.id_fasilitas) AS jumlah_fasilitas FROM Fakultas f LEFT JOIN FasilitasFakultas ff ON f.id_fakultas = ff.id_fakultas GROUP BY f.nama

Generated SQL Query: SELECT MAX(facility_count) FROM (SELECT COUNT(id_fasilitas) AS facility_count FROM FasilitasFakultas GROUP BY id_fakultas) AS subquery;

Generated SQL Query: SELECT nama FROM Fakultas WHERE id_fakultas IN (SELECT id_fakultas FROM FasilitasFakultas GROUP BY id_

In [37]:
# V1
query = text_to_sql_model.generate_v1(user_prompt)
print(query)

Rewritten Prompt :Retrieve the faculty who have used the most facilities.

Relevant Example: {'relevant_question': 'Which faculty has the most research projects? If there is a tie, show any of the tying faculty. Show the nama and total_proyek.', 'relevant_answer': 'SELECT f.nama, COUNT(p.id_proyek) AS total_proyek\n  FROM Fakultas f\n  JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas\n  GROUP BY f.nama\n  ORDER BY total_proyek DESC\n  LIMIT 1;', 'relevant_summary': "This SQL query identifies the faculty with the highest number of research projects by joining the 'Fakultas' (Faculty) and 'ProyekPenelitian' (Research Projects) tables. It counts projects per faculty and returns the faculty with the most projects, highlighting research activity distribution across faculties. The output helps academic institutions understand research productivity and allocate resources effectively. Key operations include a table JOIN, COUNT aggregation, and ORDER BY with LIMIT to identify the top fa

In [38]:
# V2
query = text_to_sql_model.generate_v2(user_prompt)
print(query)

Rewritten Prompt :Retrieve the faculty who have used the greatest number of facilities.

Relevant Example: {'relevant_question': 'List all faculties along with their total number of research projects, total publications, and number of lecturers involved in either. Show nama, total_proyek, total_publikasi, total_dosen_aktif.', 'relevant_answer': 'SELECT f.nama,\n  COUNT(DISTINCT p.id_proyek) AS total_proyek,\n  COUNT(DISTINCT pb.id_publikasi) AS total_publikasi,\n  COUNT(DISTINCT d.id_dosen) AS total_dosen_aktif\n  FROM Fakultas f\n  LEFT JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas\n  LEFT JOIN Publikasi pb ON f.id_fakultas = pb.id_fakultas\n  LEFT JOIN Dosen d ON d.id_fakultas = f.id_fakultas\n  WHERE d.id_dosen IN (\n  SELECT id_dosen FROM PenulisPublikasi\n  UNION\n  SELECT id_dosen FROM ProyekPenelitian\n  )\n  GROUP BY f.nama;', 'relevant_summary': "This SQL query provides a comprehensive research activity overview by faculty by analyzing three key metrics: research pr

In [39]:
# V3
query = text_to_sql_model.generate_v3(user_prompt)
print(query)

Rewritten Prompt :Retrieve the faculty that uses the greatest number of facilities.

Entities: ['fakultas', 'fasilitas', 'fasilitasfakultas']
Entity Embeddings: tensor([[-0.0858,  0.0030, -0.1152,  ...,  0.0751,  0.0536,  0.0192],
        [-0.0083,  0.0028, -0.0205,  ...,  0.0032,  0.1153,  0.0406],
        [-0.0313,  0.0165, -0.0624,  ...,  0.0726,  0.1328,  0.0336]],
       device='cuda:0')
Similarity Scores: [('fasilitas', 0.915180429816246), ('fasilitasfakultas', 0.8786247670650482), ('fakultas', 0.7151243090629578), ('keanggotaanklub', 0.574022650718689), ('mahasiswa', 0.552211657166481), ('jurusan', 0.5305024534463882), ('hasilujian', 0.5027420520782471), ('matakuliah', 0.4761573076248169), ('partisipasipenelitian', 0.44340774416923523), ('proyekpenelitian', 0.4387131482362747), ('peminjamanbuku', 0.4334169551730156), ('publikasi', 0.41327446699142456), ('organisasiklub', 0.36894214153289795), ('kelas', 0.32284481450915337), ('catatankehadiran', 0.3000359386205673), ('penulispubl

In [40]:
# V4
query = text_to_sql_model.generate_v4(user_prompt)
print(query)

Rewritten Prompt :Retrieve the faculty that uses the greatest number of facilities.

Sub-questions: ['1. Which facilities are used by each faculty? (Join `FasilitasFakultas` and `Fakultas` tables)', '2. Count the number of facilities used by each faculty. (Group the results by faculty ID and count the facilities)', '3. Which faculty has the highest count of facilities? (Find the maximum count and the corresponding faculty ID)', '4. What is the name of the faculty with the highest count of facilities? (Retrieve the faculty name using the faculty ID from the previous step)']
Relevant Example: {'relevant_question': 'List all faculties along with their total number of research projects, total publications, and number of lecturers involved in either. Show nama, total_proyek, total_publikasi, total_dosen_aktif.', 'relevant_answer': 'SELECT f.nama,\n  COUNT(DISTINCT p.id_proyek) AS total_proyek,\n  COUNT(DISTINCT pb.id_publikasi) AS total_publikasi,\n  COUNT(DISTINCT d.id_dosen) AS total_dose

In [41]:
# V5
query = text_to_sql_model.generate_v5(user_prompt)
print(query)

Rewritten Prompt :Retrieve the faculty that uses the greatest number of facilities.

Entities: ['fakultas', 'fasilitas', 'fasilitasfakultas']
Entity Embeddings: tensor([[-0.0858,  0.0030, -0.1152,  ...,  0.0751,  0.0536,  0.0192],
        [-0.0083,  0.0028, -0.0205,  ...,  0.0032,  0.1153,  0.0406],
        [-0.0313,  0.0165, -0.0624,  ...,  0.0726,  0.1328,  0.0336]],
       device='cuda:0')
Similarity Scores: [('fasilitas', 0.915180429816246), ('fasilitasfakultas', 0.8786247670650482), ('fakultas', 0.7151243090629578), ('keanggotaanklub', 0.574022650718689), ('mahasiswa', 0.552211657166481), ('jurusan', 0.5305024534463882), ('hasilujian', 0.5027420520782471), ('matakuliah', 0.4761573076248169), ('partisipasipenelitian', 0.44340774416923523), ('proyekpenelitian', 0.4387131482362747), ('peminjamanbuku', 0.4334169551730156), ('publikasi', 0.41327446699142456), ('organisasiklub', 0.36894214153289795), ('kelas', 0.32284481450915337), ('catatankehadiran', 0.3000359386205673), ('penulispubl